## Credit Curve Bootstrapper

The goal of the project is to build a bootstrapping procedure whose output will be a credit curve (an instance of the Python class CreditCurve). The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotations)
* The par rate of a set of Credit Default Swap (i.e. their market quotations)

The resulting class will be used to compute the market value of a set of custom CDS.

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for CDS evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [15]:
from datetime import date

pricing_date = date(2019, 10, 31)
cds_recovery = 0.4

cds_quotes = pd.read_csv("cds_quotes_proj2.csv").to_dict('records')
ois_quotes = pd.read_csv("ois_quotes.csv").to_dict('records')
cds_to_price = pd.read_csv("cds_to_price_proj2.csv").to_dict('records')

In [16]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates

pillar_dates = [pricing_date]

swaps = []

for quote in ois_quotes:
    swap = OvernightIndexSwap(
        1,
        generate_swap_dates(
            pricing_date,
            quote['maturity']
        ),
        quote['rate'])
    
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [17]:
def objective_function(x):
    
    curve = DiscountCurve(
        pricing_date, pillar_dates, x)
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [18]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates))]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates))]
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [19]:
print (result.x)
print (objective_function(x0))
print (objective_function(result.x))

[1.         0.99991167 0.99980687 0.99970619 0.9996069  0.9994978
 0.99938865 0.99927868 0.9991637  0.99902672 0.99889136 0.99874532
 0.99858427 0.99806988 0.99748298 0.99674203 0.99581479 0.98957082
 0.97816259 0.962208   0.94242529 0.91997947 0.8955561  0.8696499
 0.84280139 0.81599992 0.78929444 0.71347554 0.61196941 0.5324563
 0.46682078]
1.399018802873446
1.1153038909532198e-09


In [20]:
discount_curve = DiscountCurve(pricing_date, pillar_dates, result.x)

In [21]:
from finmarkets import generate_swap_dates, CreditDefaultSwap, CreditCurve

cds_pillar_dates = [pricing_date]

creditdefaultswaps = []
for quote in cds_quotes:
    creditdefswap = CreditDefaultSwap(
        1,
        pricing_date,
        quote['maturity']//12,
        quote['spread'])
    
    creditdefaultswaps.append(creditdefswap)
    cds_pillar_dates.append(creditdefswap.payment_dates[-1])
    
cds_pillar_dates = sorted(cds_pillar_dates)

In [22]:
def obj_function(unknown_ndps):

    curve_c = CreditCurve(cds_pillar_dates, unknown_ndps)
    sum_sq = 0.0

    for creditdefswap in creditdefaultswaps:
        sum_sq += creditdefswap.npv(discount_curve, curve_c) ** 2

    return sum_sq

In [23]:
x0_guess = [0.5 for i in range(len(cds_pillar_dates))]

bounds_credit_curve = [(0.01, 1) for i in range(len(cds_pillar_dates))]
bounds_credit_curve[0]=(1, 1)

results = minimize(obj_function, x0_guess, bounds=bounds_credit_curve)

In [24]:
print (results)
print (obj_function(x0_guess))
print (obj_function(results.x))

      fun: 7.517283377137074e-12
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.58519864e-07,  7.17196619e-07,  2.30198063e-06, -1.95940914e-06,
       -1.28630104e-06, -4.99099415e-07,  3.66227628e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 88
      nit: 10
   status: 0
  success: True
        x: array([1.        , 0.90683175, 0.80418831, 0.70888291, 0.54481268,
       0.29725113, 0.08663749])
0.5418642579596749
7.517283377137074e-12


In [25]:
credit_curve = CreditCurve(cds_pillar_dates, results.x)

In [26]:
npv_cds_to_price = []

for quote in cds_to_price:
    creditdefswapprice = CreditDefaultSwap(
        quote['nominal'],
        pricing_date,
        quote['maturity'],
        quote['spread'])
    
    npv_cds_to_price.append(creditdefswapprice.npv(discount_curve, credit_curve))

print ("Prices of CDS " + str(["{:.2f}".format(p) for p in npv_cds_to_price]))

Prices of CDS ['-214319.59', '-238059.81', '-402035.72', '-450568.74', '-596167.80', '-644700.81', '-790299.87', '-1081497.98', '-1275630.05', '-1469762.13', '-1663894.20', '-2052158.34', '-2505133.18', '-2893397.33', '-3314016.82']
